This notebook is showing how to compare different Italian documents from the insurance domain in order to extract the differences.
It uses Amazon Bedrock Claude 2.1 LLM model, the LangChain library and the FAISS lib for storing and retrieving the embeddings (vectors).
It was tested in the SageMaker Python 3 (Data Science 3.0) kernel, using the ml.c5.xlarge instance (4 vCPU + 8 GB RAM).
The insurance pdf files were taken from the public website and I don't own them.

In [2]:
! pip install --upgrade pip --quiet
##! pip install boto3==1.28.64 --quiet
! pip install boto3 --quiet
! pip install langchain --quiet
! pip install anthropic --quiet
! pip install amazon-textract-textractor --quiet
! pip install faiss-cpu transformers --quiet

In [3]:
import base64
import boto3 
import sagemaker
from pprint import pprint
#import lib.utils as utils
from IPython.display import JSON
import importlib
#importlib.reload(utils)
# importlib.reload(boto3)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
from langchain.prompts import PromptTemplate
from langchain.llms.bedrock import Bedrock
from langchain.document_loaders import AmazonTextractPDFLoader

In [5]:
bedrock_client = boto3.client("bedrock-runtime", region_name='us-east-1')
data_bucket = sagemaker.Session().default_bucket()
role = sagemaker.get_execution_role()


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [6]:
!aws s3 cp docs s3://{data_bucket}/genai --recursive --only-show-errors
!aws s3 ls s3://{data_bucket}/genai/

2023-12-15 17:14:48    5042639 3_insurances_merged.pdf
2023-12-15 17:14:48    2293574 AXA_Azienda_Protetta_2022.pdf
2023-12-15 17:14:48    1890484 Allianz-Impresa-Sicura-SI.pdf
2023-12-15 17:14:48     679494 GeneraIi_Impresa_Multigaranzia.pdf


In [7]:
from langchain.document_loaders import AmazonTextractPDFLoader
loader = AmazonTextractPDFLoader(f"s3://{data_bucket}/genai/Allianz-Impresa-Sicura-SI.pdf")
document = loader.load()
print(f"Textract extracted {len(document)} pages from the document")

Textract extracted 120 pages from the document


In [8]:
enumerate(document)


In [9]:
for index,page in enumerate(document):
    if index >= 1:
        break
    print(f"=========Page {index+1}==========")
    print(page.page_content)
    print("\n")

=========Page 1==========
Allianz
M
Assicurazione contro i danni
DIP Documento Informativo relativo al prodotto assicurativo
Compagnia: Allianz S.p.A.
Prodotto: "Impresa Sicura"
Le informazioni precontrattuali e contrattuali complete relative al prodotto sono fornite in altri documenti.
Che tipo di assicurazione è?
È un'assicurazione contro i danni che offre una serie di coperture assicurative Incendio-All Risks, Indennità da interruzione
di attività, Furto e Rapina, Responsabilità Civile dell'esercizio dell'attività, Responsabilità Civile della proprietà del
fabbricato, Tutela legale, Assistenza.
Che cosa è assicurato?
Che cosa non è assicurato?
Sezione Incendio All Risk: danni materiali e
Sezione Incendio-All Risks: fabbricato in cui
diretti alle Cose assicurate;
sono posti i locali e relativo contenuto con
Sezione Indennità da interruzione di
caratteristiche costruttive difformi rispetto a
attività: danni indiretti a seguito di Sinistro
quanto indicato nel DIP aggiuntivo Danni alla


In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                               separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
                                               chunk_overlap=200)
texts = text_splitter.split_documents(document)

for index, text in enumerate(texts):
    if index >= 3:
        break
    print(f"==== Chunk {index+1}, From Page {text.metadata['page']} ====")
    print(text.page_content)
    print("\n")

==== Chunk 1, From Page 1 ====
Allianz
M
Assicurazione contro i danni
DIP Documento Informativo relativo al prodotto assicurativo
Compagnia: Allianz S.p.A.
Prodotto: "Impresa Sicura"
Le informazioni precontrattuali e contrattuali complete relative al prodotto sono fornite in altri documenti.
Che tipo di assicurazione è?
È un'assicurazione contro i danni che offre una serie di coperture assicurative Incendio-All Risks, Indennità da interruzione
di attività, Furto e Rapina, Responsabilità Civile dell'esercizio dell'attività, Responsabilità Civile della proprietà del
fabbricato, Tutela legale, Assistenza.
Che cosa è assicurato?
Che cosa non è assicurato?
Sezione Incendio All Risk: danni materiali e
Sezione Incendio-All Risks: fabbricato in cui
diretti alle Cose assicurate;
sono posti i locali e relativo contenuto con
Sezione Indennità da interruzione di
caratteristiche costruttive difformi rispetto a
attività: danni indiretti a seguito di Sinistro
quanto indicato nel DIP aggiuntivo Danni 

In [11]:
#Here I only check what models are available in the used region (US-East-1) and what to use
br = boto3.client('bedrock')
resp = br.list_foundation_models(
    byOutputModality='EMBEDDING'
)
for model in resp['modelSummaries']:
    print(model['modelId'])

amazon.titan-e1t-medium
amazon.titan-embed-g1-text-02
amazon.titan-embed-text-v1:2:8k
amazon.titan-embed-text-v1
amazon.titan-embed-image-v1:0
amazon.titan-embed-image-v1
cohere.embed-english-v3
cohere.embed-multilingual-v3


In [12]:
from langchain.embeddings import BedrockEmbeddings

embeddings = BedrockEmbeddings(client=bedrock_client)

In [13]:
from langchain.vectorstores import FAISS

embeddings = BedrockEmbeddings(client=bedrock_client,model_id="amazon.titan-embed-text-v1")
vector_db = FAISS.from_documents(documents=texts, embedding=embeddings)

In [14]:
# to delete the vector index from the SageMaker instance memory use:
# vector_db.delete([vector_db.index_to_docstore_id[0]])

In [15]:
query = "Che differenze ci sono tra le polizze assicurative Generali e Allianz relativamente agli Eventi atmosferici. Fornisci piu dettagli possibili per le coperture, franchigie e massimal"
docs = vector_db.similarity_search(query)

In [16]:
docs

[Document(page_content='Allianz\nCondizioni di assicurazione\nL\'Impresa risponde dei danni al Macchinario posto all\'aperto, collocato all\'interno dell\'area aziendale od in spazi ove si\nsvolge l\'Attività dichiarata adiacenti ai locali dell\'Ubicazione indicata nella Scheda di Polizza, qualora lo stesso sia\ninamovibile ed ideato e destinato ad un uso esterno.\nArt. 3.9 Estensione Eventi atmosferici su Macchinario all\'aperto\nA integrazione e a parziale deroga di quanto previsto dall\'articolo 3.8 "Eventi atmosferici". sono compresi danni\nmateriali e diretti causati al Macchinario amovibile, ideato e destinato ad un uso esterno, posto all\'aperto all\'interno\ndell\'area aziendale od in spazi ove si svolge l\'Attività dichiarata ed adiacenti ai locali dell\'Ubicazione indicata nella Scheda\ndi Polizza, verificatisi per effetto di eventi atmosferici quali uragano, bufera, ciclone, tempesta, trombe d\'aria, vento e cose\nda esso trasportate o fatte crollare, grandine, pioggia, neve

In [17]:
docs = vector_db.similarity_search_with_score(query, k = 3)
docs

[(Document(page_content='Allianz\nCondizioni di assicurazione\nL\'Impresa risponde dei danni al Macchinario posto all\'aperto, collocato all\'interno dell\'area aziendale od in spazi ove si\nsvolge l\'Attività dichiarata adiacenti ai locali dell\'Ubicazione indicata nella Scheda di Polizza, qualora lo stesso sia\ninamovibile ed ideato e destinato ad un uso esterno.\nArt. 3.9 Estensione Eventi atmosferici su Macchinario all\'aperto\nA integrazione e a parziale deroga di quanto previsto dall\'articolo 3.8 "Eventi atmosferici". sono compresi danni\nmateriali e diretti causati al Macchinario amovibile, ideato e destinato ad un uso esterno, posto all\'aperto all\'interno\ndell\'area aziendale od in spazi ove si svolge l\'Attività dichiarata ed adiacenti ai locali dell\'Ubicazione indicata nella Scheda\ndi Polizza, verificatisi per effetto di eventi atmosferici quali uragano, bufera, ciclone, tempesta, trombe d\'aria, vento e cose\nda esso trasportate o fatte crollare, grandine, pioggia, nev

In [18]:
retriever = vector_db.as_retriever(search_type='mmr', search_kwargs={"k": 3})
relevant_docs = retriever.get_relevant_documents(query)   
relevant_docs

[Document(page_content='Allianz\nCondizioni di assicurazione\nL\'Impresa risponde dei danni al Macchinario posto all\'aperto, collocato all\'interno dell\'area aziendale od in spazi ove si\nsvolge l\'Attività dichiarata adiacenti ai locali dell\'Ubicazione indicata nella Scheda di Polizza, qualora lo stesso sia\ninamovibile ed ideato e destinato ad un uso esterno.\nArt. 3.9 Estensione Eventi atmosferici su Macchinario all\'aperto\nA integrazione e a parziale deroga di quanto previsto dall\'articolo 3.8 "Eventi atmosferici". sono compresi danni\nmateriali e diretti causati al Macchinario amovibile, ideato e destinato ad un uso esterno, posto all\'aperto all\'interno\ndell\'area aziendale od in spazi ove si svolge l\'Attività dichiarata ed adiacenti ai locali dell\'Ubicazione indicata nella Scheda\ndi Polizza, verificatisi per effetto di eventi atmosferici quali uragano, bufera, ciclone, tempesta, trombe d\'aria, vento e cose\nda esso trasportate o fatte crollare, grandine, pioggia, neve

In [19]:
full_context = str()
for doc in relevant_docs:
    full_context += doc.page_content+" "
    
#print(full_context.strip(".").strip())

In [20]:
br.get_foundation_model(modelIdentifier='anthropic.claude-v2:1:200k')

{'ResponseMetadata': {'RequestId': '2098c654-0ffe-4329-9e25-9770d99d3fe7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 15 Dec 2023 17:17:32 GMT',
   'content-type': 'application/json',
   'content-length': '386',
   'connection': 'keep-alive',
   'x-amzn-requestid': '2098c654-0ffe-4329-9e25-9770d99d3fe7'},
  'RetryAttempts': 0},
 'modelDetails': {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-v2:1:200k',
  'modelId': 'anthropic.claude-v2:1:200k',
  'modelName': 'Claude',
  'providerName': 'Anthropic',
  'inputModalities': ['TEXT'],
  'outputModalities': ['TEXT'],
  'responseStreamingSupported': True,
  'customizationsSupported': [],
  'inferenceTypesSupported': ['PROVISIONED']}}

In [21]:
from langchain.document_loaders import AmazonTextractPDFLoader
from langchain.llms import Bedrock
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

loader = AmazonTextractPDFLoader(f"s3://{data_bucket}/genai/Allianz-Impresa-Sicura-SI.pdf")
#loader = AmazonTextractPDFLoader(f"s3://{data_bucket}/genai/3_insurances_merged.pdf")

document1 = loader.load()



In [22]:
loader2 = AmazonTextractPDFLoader(f"s3://{data_bucket}/genai/GeneraIi_Impresa_Multigaranzia.pdf")

document2 = loader2.load()

In [23]:
# print page numbers
len(document1)

120

In [24]:
len(document2)

60

In [25]:
#query = "Che differenze ci sono tra le polizze assicurative Generali e Allianz relativamente agli Eventi atmosferici. Fornisci piu dettagli possibili per le coperture, franchigie e massimal"
#query = "che opzioni di coperatura assicurativa ci sono per il terremoto"
query = "franchigie per incendio" 
#query = "quali sono le 10 differenze le polizze assicurative indicate?"

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter1 = RecursiveCharacterTextSplitter(chunk_size=1000,
                                               separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
                                               chunk_overlap=0)

texts1 = text_splitter1.split_documents(document1)
texts2 = text_splitter1.split_documents(document2)


#Input text and the embeddings engine using the FAISS to store it the Notebook's memory
vector_db1 = FAISS.from_documents(documents=texts1, embedding=embeddings)
vector_db2 = FAISS.from_documents(documents=texts2, embedding=embeddings)



In [27]:
#docs1 = vector_db1.similarity_search_with_score(query, k = 100)
#docs2 = vector_db2.similarity_search_with_score(query, k = 100)

score = 0.90

# Use "mmr" or "similarity_score_threshold"
retriever1 = vector_db1.as_retriever(search_type='mmr', search_kwargs={"k": 30, "score_threshold" : score})
retriever2 = vector_db2.as_retriever(search_type='mmr', search_kwargs={"k": 30, "score_threshold" : score})


relevant_docs1 = retriever1.get_relevant_documents(query)
relevant_docs2 = retriever2.get_relevant_documents(query)

#relevant_docs2

In [28]:
relevant_docs1

[Document(page_content="Sezione Incendio-All Risks)\nSezione\nL'Impresa indennizza danni materiali e diretti in conseguenza di Furto e Rapina, tentati\no\nFurto e Rapina\nconsumati, compresi i guasti cagionati dai ladri e danni derivanti da atti vandalici\ncommessi durante il Furto e/o la Rapina o nel tentativo di commetterli; l'Impresa\nindennizza inoltre una serie di spese, oneri e onorari resesi necessari a seguito di Furto e\nRapina.\nSezione\nL'Impresa tiene indenne l'Assicurato, quale civilmente responsabile ai sensi di legge, di\nResponsabilità Civile\nquanto sia tenuto a pagare a titolo di risarcimento (capitale, interessi, spese) per danni\ndell'esercizio dell'attività\ninvolontariamente cagionati a terzi per morte, lesioni personali, distruzione o\nDIPA-454-ed.21102023\nPag. 1 di 41", metadata={'source': 's3://sagemaker-us-east-1-174976546647/genai/Allianz-Impresa-Sicura-SI.pdf', 'page': 5}),
 Document(page_content="franchigie/scoperti eventualmente previsti nella Scheda di P

In [29]:
relevant_docs2

[Document(page_content="c) il costruttore non abbia cessato la fabbricazione dell'impianto o dell'apparecchio danneggiato o\ndistrutto, oppure questo sia ancora disponibile o siano disponibili i pezzi di ricambio.\nCondizioni di assicurazione\nEdizione 01.06.2017\nSezione Incendio - Pagina 23\ndi\n60\nGeneraimpresa\nmod. PMI99/05", metadata={'source': 's3://sagemaker-us-east-1-174976546647/genai/GeneraIi_Impresa_Multigaranzia.pdf', 'page': 23}),
 Document(page_content="Impossessamento della cosa mobile altrui, sottraendola a chi la detiene, al fine\ndi trarne ingiusto profitto per sé o per altri.\nIncendio\nCombustione, con fiamma, di beni materiali al di fuori di appropriato focolare,\nche può autoestendersi e propagarsi.\nInfiammabili\ni gas combustibili come, ad esempio, l'acetilene, il metano, l'etano, ecc.;\nle sostanze con punto di infiammabilità inferiore a 55° C (quali ad esempio:\nbenzina, alcole, vernici alla nitrocellulosa. i più comuni solventi e diluenti ecc.).\nIl punto d

In [30]:
full_context1 = str()
full_context2 = str()

for doc in relevant_docs1:
    full_context1 += doc.page_content+" "
    
for doc in relevant_docs2:
    full_context2 += doc.page_content+" "
    


In [31]:
question = "per le polizze Allianz e Generali, elenca tutti i dettali in modo specifico ed evidenzia le differenze per la copertura solo in caso di incendio, con riferimento solo alla franchigia"

In [32]:
template = """

Human: Assume you are an insurance broker. Answer the {question} using the provided text. Skip any preamble text and reasoning and give just the answer. Answer in the same language as the question.


<question>{question}</question>
<text>{allianz}</text>
<text>{generali}</text>
<answer>


Assistant:"""

prompt = PromptTemplate(template=template, input_variables=["allianz","generali","question"])
bedrock_llm = Bedrock(client=bedrock_client,
                      model_id="anthropic.claude-v2:1",
                      model_kwargs={'max_tokens_to_sample': 20000,
                                    'temperature': 0.4,
                                    'top_k': 500,
                                    'top_p': 0.6,
                                    'stop_sequences': ['Human:']},
                     ) # anthropic.claude-v2:1:200k

#Add verbose=True to the LLMChain for debug
llm_chain = LLMChain(prompt=prompt, llm=bedrock_llm)

answer = llm_chain.run(allianz = full_context1, generali = full_context2, question = question )
print(answer.strip())


Ecco i dettagli specifici per le polizze Allianz e Generali in relazione alla copertura per incendio, con riferimento solo alla franchigia:

Allianz:
- Non sono previste franchigie specifiche per la copertura incendio. Si applica l'eventuale franchigia frontale indicata in scheda di polizza per ogni sinistro.

Generali: 
- Per i danni derivanti da incendio, esplosione e scoppio opera uno scoperto del 10%, con il minimo di 250 euro per sinistro.

Differenze:
- Allianz: nessuna franchigia specifica per incendio. Si applica l'eventuale franchigia frontale.  
- Generali: per incendio opera uno scoperto del 10% con franchigia minima di 250 euro.

</answer>


In [33]:
#Answer the {question} using the provided text. Skip any preamble text and reasoning and give just the answer. Answer in the same language as the question.